In [2]:
import numpy as np
import pandas as pd

In [5]:
data=pd.read_csv(r'C:\Users\anjan\Downloads\P16-Artificial-Neural-Networks\Part 1 - Artificial Neural Networks\Churn_Modelling.csv')

In [6]:
X = data.iloc[:, 3:-1].values
y = data.iloc[:, -1].values

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [12]:
# Define the neural network architecture
input_size = 2
hidden_size = 3
output_size = 1

# Define the number of individuals in the population
population_size = 10

# Define the maximum number of generations
max_generations = 100

# Define the selection pressure
tournament_size = 2

# Define the mutation rate
mutation_rate = 0.1

# Define the fitness function
def fitness_function(weights):
    # Define the neural network with the given weights
    w1 = np.reshape(weights[:hidden_size * input_size], (hidden_size, input_size))
    b1 = np.reshape(weights[hidden_size * input_size:hidden_size * input_size + hidden_size], (hidden_size, 1))
    w2 = np.reshape(weights[hidden_size * input_size + hidden_size:hidden_size * input_size + hidden_size + output_size * hidden_size], (output_size, hidden_size))
    b2 = np.reshape(weights[hidden_size * input_size + hidden_size + output_size * hidden_size:], (output_size, 1))
    z1 = np.dot(w1, X.T) + b1
    a1 = np.tanh(z1)
    z2 = np.dot(w2, a1) + b2
    a2 = np.tanh(z2)

    # Compute the mean squared error
    mse = np.mean((a2 - y) ** 2)

    return -mse



In [ ]:
# Initialize the population
population = np.random.uniform(-1, 1, (population_size, input_size * hidden_size + hidden_size + output_size * hidden_size + output_size))

# Initialize the individual knowledge
individual_knowledge = np.copy(population)

# Initialize the cultural knowledge
cultural_knowledge = np.zeros((output_size * hidden_size + output_size + hidden_size * input_size + hidden_size))

# Iterate over the generations
for generation in range(max_generations):

    # Evaluate the fitness of each individual in the population
    fitness = np.zeros(population_size)
    for i in range(population_size):
        fitness[i] = fitness_function(population[i])
    
    # Update the individual knowledge
    for i in range(population_size):
        if fitness[i] > fitness_function(individual_knowledge[i]):
            individual_knowledge[i] = np.copy(population[i])

    # Update the cultural knowledge
    best_individual = population[np.argmax(fitness)]
    if fitness_function(best_individual) > fitness_function(cultural_knowledge):
        cultural_knowledge = np.copy(best_individual)

    # Create a new population
    new_population = np.zeros((population_size, input_size * hidden_size + hidden_size + output_size * hidden_size + output_size))

    # Perform selection, crossover, and mutation
    for i in range(population_size):
        parent1 = population[np.random.randint(population_size)]
        parent2 = population[np.random.randint(population_size)]
        tournament = np.argsort(np.random.rand(tournament_size) * fitness[np.random.randint(population_size, size=tournament_size)])
        parent1 = tournament[0]
        parent2 = tournament[1]
        offspring = np.zeros(input_size * hidden_size + hidden_size + output_size * hidden_size + output_size)
        for j in range(input_size * hidden_size + hidden_size + output_size * hidden_size + output_size):
            if np.random.rand() < mutation_rate:
                offspring[j] = np.random.uniform(-1, 1)
            else:
                if np.random.rand() < 0.5:
                    offspring[j] = individual_knowledge[parent]
